# Notebook 02: COG Enrichment Analysis by Lifestyle

**Project**: Lifestyle-Based COG Stratification

**Goal**: Compare COG functional enrichment patterns (core vs accessory) between host-associated and free-living species.

**Hypotheses**:
- H1: Host-associated bacteria have higher defense (COG V) enrichment in accessory genes
- H2: Free-living bacteria have greater metabolic diversity (COG E,G,C,P,I) in accessory genes
- H3: Host-associated bacteria have smaller core genome fractions

**Input**: `../data/species_lifestyle_classification.csv` (from Notebook 01)

**Output**: `../data/cog_enrichment_by_lifestyle.csv`, `../figures/enrichment_heatmap.png`

In [ ]:
spark = get_spark_session()
import pandas as pd
import numpy as np

In [ ]:
# Load species lifestyle classification from Notebook 01
species_df = pd.read_csv('../data/species_lifestyle_classification.csv')
print(f"Total target species: {len(species_df)}")
print(species_df['species_lifestyle'].value_counts())

species_list = species_df['gtdb_species_clade_id'].tolist()
print(f"\nSpecies IDs loaded: {len(species_list)}")

## 1. COG Distribution: Core vs Accessory per Species

For each target species, query `gene_cluster` joined with `eggnog_mapper_annotations` to get
COG category counts by core/accessory status.

**Safety**: Always filter by `gtdb_species_clade_id` — these are large tables.

In [ ]:
# Process species in batches to manage memory
# Batch size depends on species pangenome size; 10-20 species per batch is safe
BATCH_SIZE = 20

all_results = []

for i in range(0, len(species_list), BATCH_SIZE):
    batch = species_list[i:i + BATCH_SIZE]
    # Escape single quotes in species IDs (shouldn't be needed but safe)
    in_clause = "', '".join(batch)
    
    batch_result = spark.sql(f"""
        SELECT
            gc.gtdb_species_clade_id,
            gc.is_core,
            ann.COG_category,
            COUNT(*) as cluster_count
        FROM kbase_ke_pangenome.gene_cluster gc
        JOIN kbase_ke_pangenome.eggnog_mapper_annotations ann
            ON gc.gene_cluster_id = ann.query_name
        WHERE gc.gtdb_species_clade_id IN ('{in_clause}')
            AND ann.COG_category IS NOT NULL
            AND ann.COG_category != '-'
        GROUP BY gc.gtdb_species_clade_id, gc.is_core, ann.COG_category
        ORDER BY gc.gtdb_species_clade_id, gc.is_core, cluster_count DESC
    """)
    
    all_results.append(batch_result.toPandas())
    print(f"Batch {i // BATCH_SIZE + 1}/{(len(species_list) + BATCH_SIZE - 1) // BATCH_SIZE}: "
          f"{len(batch)} species, {len(all_results[-1])} rows")

cog_raw = pd.concat(all_results, ignore_index=True)
print(f"\nTotal COG rows: {len(cog_raw):,}")
print(f"Species with data: {cog_raw['gtdb_species_clade_id'].nunique()}")
cog_raw.head(10)

## 2. Handle Multi-Character COG Categories

Some genes have composite COG categories (e.g., "LV", "EGP").
Split these so each character is counted separately, then normalize.

In [ ]:
# Check how common multi-character COG categories are
cog_raw['cog_len'] = cog_raw['COG_category'].str.len()
print("COG category length distribution:")
print(cog_raw.groupby('cog_len')['cluster_count'].sum())
print(f"\nMulti-char fraction: {cog_raw[cog_raw['cog_len'] > 1]['cluster_count'].sum() / cog_raw['cluster_count'].sum():.1%}")

In [ ]:
# Split multi-character COG categories into individual characters
# Each character gets the full count (gene is counted in each category)
rows = []
for _, row in cog_raw.iterrows():
    for char in row['COG_category']:
        if char.isalpha():  # Skip any non-letter characters
            rows.append({
                'gtdb_species_clade_id': row['gtdb_species_clade_id'],
                'is_core': row['is_core'],
                'COG_category': char,
                'cluster_count': row['cluster_count']
            })

cog_split = pd.DataFrame(rows)

# Aggregate after splitting
cog_agg = cog_split.groupby(
    ['gtdb_species_clade_id', 'is_core', 'COG_category']
)['cluster_count'].sum().reset_index()

print(f"After splitting: {len(cog_agg):,} rows")
print(f"Unique COG categories: {sorted(cog_agg['COG_category'].unique())}")

## 3. Calculate COG Proportions and Enrichment Scores

For each species, compute:
- Proportion of each COG category within core genes
- Proportion of each COG category within accessory genes (is_core = 0)
- Enrichment score: `(prop_accessory - prop_core) / prop_core`
  - Positive = enriched in accessory, Negative = enriched in core

In [ ]:
# Calculate proportions within core and accessory for each species
# is_core: 1 = core, 0 = accessory (includes auxiliary + singleton)

# Total counts per species per compartment
totals = cog_agg.groupby(
    ['gtdb_species_clade_id', 'is_core']
)['cluster_count'].sum().reset_index().rename(columns={'cluster_count': 'total'})

# Join totals and compute proportions
cog_props = cog_agg.merge(totals, on=['gtdb_species_clade_id', 'is_core'])
cog_props['proportion'] = cog_props['cluster_count'] / cog_props['total']

# Pivot to get core and accessory proportions side by side
core_props = cog_props[cog_props['is_core'] == 1][['gtdb_species_clade_id', 'COG_category', 'proportion']].rename(
    columns={'proportion': 'prop_core'}
)
acc_props = cog_props[cog_props['is_core'] == 0][['gtdb_species_clade_id', 'COG_category', 'proportion']].rename(
    columns={'proportion': 'prop_accessory'}
)

enrichment = core_props.merge(acc_props, on=['gtdb_species_clade_id', 'COG_category'], how='outer')
enrichment = enrichment.fillna(0)

# Enrichment score: (accessory - core) / core
# Use a small pseudocount to avoid division by zero
PSEUDO = 1e-4
enrichment['enrichment_score'] = (
    (enrichment['prop_accessory'] - enrichment['prop_core'])
    / (enrichment['prop_core'] + PSEUDO)
)

print(f"Enrichment table: {len(enrichment):,} rows")
enrichment.head(10)

In [ ]:
# Add lifestyle classification
enrichment = enrichment.merge(
    species_df[['gtdb_species_clade_id', 'species_lifestyle', 'phylum', 'GTDB_species', 'no_genomes']],
    on='gtdb_species_clade_id',
    how='inner'
)

print(f"Species with enrichment + lifestyle: {enrichment['gtdb_species_clade_id'].nunique()}")
print(enrichment.groupby('species_lifestyle')['gtdb_species_clade_id'].nunique())

## 4. Compare Enrichment Between Lifestyles

For each COG category, compare the distribution of enrichment scores between host-associated and free-living species.

In [ ]:
from scipy import stats

# COG category descriptions for reference
cog_descriptions = {
    'J': 'Translation, ribosomal',
    'A': 'RNA processing',
    'K': 'Transcription',
    'L': 'Replication, recombination, repair',
    'B': 'Chromatin structure',
    'D': 'Cell cycle control',
    'Y': 'Nuclear structure',
    'V': 'Defense mechanisms',
    'T': 'Signal transduction',
    'M': 'Cell wall/membrane',
    'N': 'Cell motility',
    'Z': 'Cytoskeleton',
    'W': 'Extracellular structures',
    'U': 'Intracellular trafficking',
    'O': 'Post-translational modification',
    'X': 'Mobilome',
    'C': 'Energy production',
    'G': 'Carbohydrate metabolism',
    'E': 'Amino acid metabolism',
    'F': 'Nucleotide metabolism',
    'H': 'Coenzyme metabolism',
    'I': 'Lipid metabolism',
    'P': 'Inorganic ion transport',
    'Q': 'Secondary metabolites',
    'R': 'General function prediction',
    'S': 'Function unknown'
}

# Statistical comparison for each COG category
results = []
for cog in sorted(enrichment['COG_category'].unique()):
    host = enrichment[
        (enrichment['COG_category'] == cog) & (enrichment['species_lifestyle'] == 'host_associated')
    ]['enrichment_score']
    free = enrichment[
        (enrichment['COG_category'] == cog) & (enrichment['species_lifestyle'] == 'free_living')
    ]['enrichment_score']
    
    if len(host) >= 5 and len(free) >= 5:
        stat, pval = stats.mannwhitneyu(host, free, alternative='two-sided')
        results.append({
            'COG_category': cog,
            'description': cog_descriptions.get(cog, '?'),
            'n_host': len(host),
            'n_free': len(free),
            'median_host': host.median(),
            'median_free': free.median(),
            'diff_median': host.median() - free.median(),
            'U_statistic': stat,
            'p_value': pval
        })

stats_df = pd.DataFrame(results)

# Multiple testing correction (Benjamini-Hochberg)
from scipy.stats import false_discovery_control
if len(stats_df) > 0:
    # BH correction
    stats_df = stats_df.sort_values('p_value')
    n_tests = len(stats_df)
    stats_df['rank'] = range(1, n_tests + 1)
    stats_df['p_adjusted'] = stats_df['p_value'] * n_tests / stats_df['rank']
    stats_df['p_adjusted'] = stats_df['p_adjusted'].clip(upper=1.0)
    # Ensure monotonicity
    stats_df['p_adjusted'] = stats_df['p_adjusted'][::-1].cummin()[::-1]
    stats_df['significant'] = stats_df['p_adjusted'] < 0.05

print("\n=== COG Enrichment: Host-Associated vs Free-Living ===")
print("Positive diff_median = more enriched in accessory for host-associated\n")
print(stats_df.to_string(index=False))

## 5. Hypothesis-Specific Tests

In [ ]:
# H1: Defense (V) enrichment in host-associated vs free-living
if 'V' in stats_df['COG_category'].values:
    v_row = stats_df[stats_df['COG_category'] == 'V'].iloc[0]
    print("=== H1: Defense (V) Enrichment ===")
    print(f"Host-associated median enrichment: {v_row['median_host']:.4f}")
    print(f"Free-living median enrichment:     {v_row['median_free']:.4f}")
    print(f"Difference:                        {v_row['diff_median']:.4f}")
    print(f"p-value (adjusted):                {v_row['p_adjusted']:.4e}")
    print(f"Significant:                       {v_row['significant']}")
else:
    print("COG category V not found in results")

print()

# H2: Metabolic diversity (E, G, C, P, I) in free-living vs host-associated
metabolic_cats = ['E', 'G', 'C', 'P', 'I']
print("=== H2: Metabolic Enrichment (E, G, C, P, I) ===")
for cat in metabolic_cats:
    if cat in stats_df['COG_category'].values:
        row = stats_df[stats_df['COG_category'] == cat].iloc[0]
        sig = '*' if row['significant'] else ''
        print(f"  {cat} ({row['description'][:25]:25s}): "
              f"host={row['median_host']:+.4f}, free={row['median_free']:+.4f}, "
              f"p_adj={row['p_adjusted']:.4e} {sig}")

In [ ]:
# H3: Core genome fraction comparison
host_species = species_df[species_df['species_lifestyle'] == 'host_associated']
free_species = species_df[species_df['species_lifestyle'] == 'free_living']

host_species = host_species.copy()
free_species = free_species.copy()
host_species['core_fraction'] = host_species['no_core'] / host_species['no_gene_clusters']
free_species['core_fraction'] = free_species['no_core'] / free_species['no_gene_clusters']

stat_h3, pval_h3 = stats.mannwhitneyu(
    host_species['core_fraction'].dropna(),
    free_species['core_fraction'].dropna(),
    alternative='two-sided'
)

print("=== H3: Core Genome Fraction ===")
print(f"Host-associated: median={host_species['core_fraction'].median():.4f}, "
      f"mean={host_species['core_fraction'].mean():.4f}, n={len(host_species)}")
print(f"Free-living:     median={free_species['core_fraction'].median():.4f}, "
      f"mean={free_species['core_fraction'].mean():.4f}, n={len(free_species)}")
print(f"Mann-Whitney U: {stat_h3:.1f}, p-value: {pval_h3:.4e}")

## 6. Quick Visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Heatmap: median enrichment score per COG category per lifestyle
pivot = enrichment.groupby(
    ['species_lifestyle', 'COG_category']
)['enrichment_score'].median().reset_index()

pivot_wide = pivot.pivot(index='COG_category', columns='species_lifestyle', values='enrichment_score')
pivot_wide = pivot_wide.reindex(sorted(pivot_wide.index))

fig, ax = plt.subplots(figsize=(8, 10))
sns.heatmap(
    pivot_wide, annot=True, fmt='.3f', cmap='RdBu_r', center=0,
    xticklabels=True, yticklabels=True, ax=ax
)
ax.set_title('Median COG Enrichment Score (Accessory vs Core)\nby Bacterial Lifestyle')
ax.set_ylabel('COG Category')
ax.set_xlabel('Lifestyle')
plt.tight_layout()
plt.savefig('../figures/enrichment_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: ../figures/enrichment_heatmap.png")

In [ ]:
# Save full enrichment data
enrichment.to_csv('../data/cog_enrichment_by_lifestyle.csv', index=False)
stats_df.to_csv('../data/cog_lifestyle_stats.csv', index=False)
print(f"Saved enrichment data: {len(enrichment):,} rows")
print(f"Saved stats: {len(stats_df)} COG categories tested")

## Results

Record findings here after running:

- **H1 (Defense V enrichment)**: ___
- **H2 (Metabolic diversity)**: ___
- **H3 (Core fraction)**: ___
- Most significant COG categories: ___
- Any unexpected patterns? ___
- Annotation coverage concerns? ___